<a href="https://colab.research.google.com/github/haaden/MSDS_498/blob/main/MSDS498_FINBERT_ML_Evaluate_berttopic%26top2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import pandas as pd

In [ ]:
import os
import random

import pickle 
import re,string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

import gensim
from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel
from gensim import corpora
from gensim import similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import corpora
from gensim.models import LsiModel,LdaModel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_doc(doc): 
    #split document into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]         
    # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]
    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    doc= ' '.join(doc)
    return doc

In [ ]:
pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.1 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [ ]:
# tested in transformers==4.18.0 
import transformers
transformers.__version__

'4.29.2'

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg',truncation=True, max_length=512)

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer, truncation=True, max_length=512)

In [ ]:
nlp("hello how are you doing")

[{'label': 'None', 'score': 0.7497324347496033}]

In [ ]:
traindata= pd.read_pickle('traindata.pkl')

In [ ]:
traindata['predict']= [nlp(i) for i in traindata['Document']]

In [ ]:
ls = traindata['predict'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
traindata = pd.concat((traindata,ls), axis =1)

In [ ]:
traindata[(traindata['CustomName']!=traindata['label']) & (traindata['Probability']>0.80) ] #& (traindata['score']>0.80)]#["Name"].unique()

In [ ]:
testdf = traindata[(traindata['CustomName']!=traindata['label'])]# & (traindata['Probability']>0.80) ]

In [ ]:
testdf.groupby(['Name','Topic']).count()

,,Document,CustomName,Top_n_words,Probability,Representative_document,predict,label,score
Name,Topic,,,,,,,,
0_our environmental_environmental impact_the environment_our sustainability,0,516,516,516,516,516,516,516,516
10_our dei_inclusion dei_dei strategy_diversity equity,10,62,62,62,62,62,62,62,62
11_more races_hispanic or_black or_or latino,11,39,39,39,39,39,39,39,39
1_climate change_of climate_physical risks_extreme weather,1,197,197,197,197,197,197,197,197
2_our esg_esg strategy_esg goals_esg issues,2,932,932,932,932,932,932,932,932
3_personal information_our privacy_data protection_data privacy,3,798,798,798,798,798,798,798,798
4_and inclusion_diversity equity_an inclusive_equity and,4,106,106,106,106,106,106,106,106
5_our safety_safety is_of safety_safety culture,5,24,24,24,24,24,24,24,24
6_the pandemic_covid19 pandemic_the covid19_pandemic we,6,51,51,51,51,51,51,51,51


In [ ]:
traindf= traindata[~traindata['Topic'].isin([0,2,3,9])]

In [ ]:
traindf[(traindata['CustomName']!=traindata['label'])]# & (traindf['Probability']>.90)]

<ipython-input-72-800560401b00>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  traindf[(traindata['CustomName']!=traindata['label'])]# & (traindf['Probability']>.90)]


,Document,Topic,Name,CustomName,Top_n_words,Probability,Representative_document,predict,label,score
300,climate impacts of business mix amount of back...,1,1_climate change_of climate_physical risks_ext...,Environmental,climate change - of climate - physical risks -...,1.000000,False,"[{'label': 'None', 'score': 0.9940038323402405}]",None,0.994004
346,our employees take part in annual code of cond...,8,8_the code_code of_our code_business conduct,Governance,the code - code of - our code - business condu...,1.000000,False,"[{'label': 'Social', 'score': 0.94343233108520...",Social,0.943432
464,"as shown in table 3, at right, aig made meanin...",11,11_more races_hispanic or_black or_or latino,Social,more races - hispanic or - black or - or latin...,0.601126,False,"[{'label': 'None', 'score': 0.8156286478042603}]",None,0.815629
544,"in 2020, the covid-19 pandemic and two hurrica...",6,6_the pandemic_covid19 pandemic_the covid19_pa...,Social,the pandemic - covid19 pandemic - the covid19 ...,0.970958,False,"[{'label': 'Environmental', 'score': 0.9899946...",Environmental,0.989995
553,"in august 2020, our headquarters in deridder, ...",1,1_climate change_of climate_physical risks_ext...,Environmental,climate change - of climate - physical risks -...,1.000000,False,"[{'label': 'Social', 'score': 0.52152460813522...",Social,0.521525
...,...,...,...,...,...,...,...,...,...,...
108063,at xperi we prioritize diversity and seek to c...,4,4_and inclusion_diversity equity_an inclusive_...,Social,and inclusion - diversity equity - an inclusiv...,1.000000,False,"[{'label': 'Environmental', 'score': 0.9230031...",Environmental,0.923003
108596,"as part of our employee onboarding program, we...",8,8_the code_code of_our code_business conduct,Governance,the code - code of - our code - business condu...,1.000000,False,"[{'label': 'Social', 'score': 0.88486850261688...",Social,0.884869
108804,factors that could cause these differences inc...,1,1_climate change_of climate_physical risks_ext...,Environmental,climate change - of climate - physical risks -...,0.922750,False,"[{'label': 'None', 'score': 0.9698266983032227}]",None,0.969827
108929,our commitment to dei starts at the top with t...,10,10_our dei_inclusion dei_dei strategy_diversit...,Social,our dei - inclusion dei - dei strategy - diver...,0.467838,False,"[{'label': 'Environmental', 'score': 0.7796066...",Environmental,0.779607


In [ ]:
590/5196

0.11354888375673595

In [ ]:
#traindf.to_pickle('traineddf.pkl')
traindf= pd.read_pickle('traineddf.pkl')

In [ ]:
testlist= clean_doc("total e-waste processed 804,687 970,526 1,003,069 total e-waste recovered 798,006 965,931 997,539 recycling 736,061 897,337 964,608 resale 51,909 54,958 32,931 return to service 10,036 13,636 total e-waste sent to landfill 6,681 4,595 5,530 landfill diversion rate for e-waste <1% <1% <1% footnotes 1 includes electricity only")

In [ ]:
nlp(final_processed_text(testlist))

In [ ]:
traindf.columns

Index(['Document', 'Topic', 'Name', 'CustomName', 'Top_n_words', 'Probability',
       'Representative_document', 'predict', 'label', 'score'],
      dtype='object')

In [ ]:
traindf.columns

Index(['Document', 'Topic', 'Name', 'CustomName', 'Top_n_words', 'Probability',
       'Representative_document', 'predict', 'label', 'score'],
      dtype='object')

In [ ]:
dfdata= traindf[['Document', 'Topic', 'Name', 'CustomName', 'Top_n_words', 'Probability']].copy()

In [ ]:
col         = 'CustomName'
conditions  = [ dfdata[col] == 'Environmental', (dfdata[col] =='Social') & (dfdata[col]== 'Governance')]
choices     = [ 0, 1, 2 ]
    
dfdata["label"] = np.where(dfdata[col] == 'Environmental', 0,(np.where(dfdata[col]=='Social',1,2)))


In [ ]:
#msk = np.random.rand(len(dfdata)) < 0.8

#train = dfdata[msk]

#test = dfdata[~msk]

In [ ]:
test_df= pd.read_pickle('testdftopic2vec_v1.pkl')

In [ ]:
test_df.columns

Index(['doc', 'score', 'id', 'label'], dtype='object')

In [ ]:
pip install swifter

In [ ]:
import swifter

In [ ]:
test_df["Y_label"] = np.where(test_df[col] == 'E', 0,(np.where(test_df[col]=='S',1,2)))

In [ ]:
test_df.columns

In [ ]:
test_df['clean_doc']= test_df['doc'].apply(lambda x: final_processed_text(clean_doc(x)))

,doc,score,id,label,Y_label,predict,customlabel,clean_doc
0,we divert as much e-waste as possible from lan...,0.825526,99981,E,0,"[{'label': 'Environmental', 'score': 0.9904370...",Environmental,divert ewaste possible landfills reusing respo...
1,one of our most successful environmental initi...,0.801161,13240,E,0,"[{'label': 'Environmental', 'score': 0.9921378...",Environmental,successful environmental initiatives waste div...
2,we estimate that about 48% of our waste is div...,0.799703,96919,E,0,"[{'label': 'Environmental', 'score': 0.9913393...",Environmental,estimate waste diverted recycling stores distr...
3,we are working to lessen our environmental imp...,0.797259,31693,E,0,"[{'label': 'Environmental', 'score': 0.9923053...",Environmental,working lessen environmental impact initiative...
4,wba continues to prioritize waste management a...,0.795229,105887,E,0,"[{'label': 'Environmental', 'score': 0.9899513...",Environmental,continues prioritize waste management works bu...


In [ ]:
test_df['predict']= test_df['doc'].swifter.apply(nlp)

Pandas Apply:   0%|          | 0/17000 [00:00<?, ?it/s]

In [ ]:
test_df= test_df.reset_index()

In [ ]:
test_df['predict_cleandoc']= test_df['clean_doc'].swifter.apply(nlp)

Pandas Apply:   0%|          | 0/17000 [00:00<?, ?it/s]

In [ ]:
test_df.columns

Index(['index', 'doc', 'score', 'id', 'label', 'Y_label', 'predict',
       'customlabel', 'clean_doc', 'predict_cleandoc'],
      dtype='object')

In [ ]:
test_df.to_pickle("testdata_clean_nlp.pkl")

In [ ]:
from google.colab import files
files.download('testdata_clean_nlp.pkl') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tls = test_df['predict_cleandoc'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
tls.head()

,label,score
0,Environmental,0.988906
1,Environmental,0.990335
2,Environmental,0.990742
3,Environmental,0.991671
4,Environmental,0.987630


In [ ]:
tls.coloums=['label_p','score']

<ipython-input-209-ab2694dbb9fa>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tls.coloums=['label_p','score']


In [ ]:
tls.coloums

['label_p', 'score']

In [ ]:
test_df['label'].unique()

array(['E', 'S', 'G'], dtype=object)

In [ ]:
col='label'
test_df["customlabel"] = np.where(test_df[col] == 'E', 'Environmental',(np.where(test_df[col]=='S','Social','Governance')))

In [ ]:
testdata = pd.concat((test_df,tls), axis =1)

In [ ]:
testdata.columns

Index(['index', 'doc', 'score', 'id', 'label', 'Y_label', 'predict',
       'customlabel', 'clean_doc', 'predict_cleandoc', 'label', 'score'],
      dtype='object')

In [ ]:
testdata.columns= ['index','doc', 'score', 'id', 'label', 'Y_label', 'predict', 'customlabel','clean_doc', 'predict_cleandoc',
       'label_p', 'score_p']

In [ ]:
test_df[test_df['score']>0.7].shape

(1599, 7)

In [ ]:
cleantest_df.iloc[351]['doc']

'total e-waste processed 804,687 970,526 1,003,069 total e-waste recovered 798,006 965,931 997,539 recycling 736,061 897,337 964,608 resale 51,909 54,958 32,931 return to service 10,036 13,636 total e-waste sent to landfill 6,681 4,595 5,530 landfill diversion rate for e-waste <1% <1% <1% footnotes 1 includes electricity only.'

In [ ]:
tokenizer("total e-waste processed 804,687 970,526 1,003,069 total e-waste recovered 798,006 965,931 997,539 recycling 736,061 897,337 964,608 resale 51,909 54,958 32,931 return to service 10,036 13,636 total e-waste sent to landfill 6,681 4,595 5,530 landfill diversion rate for e-waste <1% <1% <1% footnotes 1 includes electricity only")

{'input_ids': [3, 87, 2178, 85, 2520, 5234, 2165, 712, 585, 4007, 767, 5706, 487, 585, 2906, 727, 428, 585, 15942, 585, 10373, 702, 87, 2178, 85, 2520, 4526, 4484, 711, 585, 18035, 727, 5101, 651, 585, 5018, 584, 5192, 767, 585, 3278, 702, 6979, 4992, 727, 585, 10373, 584, 4789, 767, 585, 20676, 5101, 712, 585, 1594, 711, 5770, 3559, 585, 1731, 702, 3639, 585, 4689, 711, 2909, 585, 5018, 584, 296, 9, 205, 685, 585, 16354, 727, 1412, 585, 4182, 727, 87, 2178, 85, 2520, 6433, 9, 9253, 1146, 585, 4007, 584, 973, 585, 3812, 651, 898, 585, 17141, 9253, 6043, 102, 14, 2178, 85, 2520, 30838, 428, 739, 30838, 428, 739, 30838, 428, 739, 13495, 428, 377, 2800, 220, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
testdata[(testdata['customlabel']!= testdata['label_p'])& (testdata['score_p']>0.7)]

In [ ]:
testdata.to_pickle('testdata_nlpbert.pkl')

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = dfdata["Document"].tolist()
y = dfdata["label"].tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.5,random_state=101)

In [ ]:
X_test= test_df['doc'].tolist()
Y_test= test_df['Y_label'].tolist()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()

In [ ]:
tv= TfidfVectorizer()

In [ ]:
x_train = cv.fit_transform(X_train)
x_test = cv.transform(X_test)

In [ ]:
xtrain= tv.fit_transform(X_train)
xtest = tv.fit(X_test)

In [ ]:
xtrain.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
def classifiers(x, y, model_type, cv = 3):
    
    #this function is to fit 3 different model scenarios.  Support vector machines, logistic regressions, naive bayes.
    #svm = Support vector machin
    #logistic = Logistic regression
    #naive_bayes = Naive Bayes Multinomial
    
    #can define cv value for cross validation.
    
    #function returns the train test split scores of each model.
    
    if model_type == 'svm':
        print("svm")
        model = SVC()

    elif model_type == 'logistic':
        print("logistic")
        model = LogisticRegression()

    elif model_type == 'naive_bayes':
        print("naive_bayes")
        model = MultinomialNB()
    
    elif model_type == 'randomforest':
        print("randomforest")
        model = RandomForestClassifier()
    
    model.fit(x,y)

    
   

    
    predictions = model.predict(x)
    accy = accuracy_score(y, predictions)
    
    return model , accy

In [ ]:
sv, acc = classifiers(x_train,y_train,  'svm')

svm


In [ ]:
acc

0.9903772132409546

In [ ]:
y_pred = sv.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
print("Accuracy : ", round(accuracy_score(Y_test, y_pred)*100, 2), "%")

Accuracy :  52.09 %


In [ ]:
print("Confusion Matrix")
print('\n')
print(confusion_matrix(y_test, y_pred))
print('\n')
print("Classification Report")
print('\n')
print(classification_report(y_test, y_pred))

Confusion Matrix


[[ 364   65    0]
 [   3 1447    1]
 [   0   15  184]]


Classification Report


              precision    recall  f1-score   support

           0       0.99      0.85      0.91       429
           1       0.95      1.00      0.97      1451
           2       0.99      0.92      0.96       199

    accuracy                           0.96      2079
   macro avg       0.98      0.92      0.95      2079
weighted avg       0.96      0.96      0.96      2079



In [ ]:
test  = cv.transform(["ncome. Income, or how much money we have, can determine where we live, food"])

In [ ]:
sv.predict(test)

array([1])

Naivebayes

In [ ]:
nv, nv_acc= classifiers(x_train,y_train,  'naive_bayes')

naive_bayes


In [ ]:
nv_acc

0.993841416474211

In [ ]:
y_pred = nv.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
print("Accuracy : ", round(accuracy_score(Y_test, y_pred)*100, 2), "%")

Accuracy :  64.25 %


In [ ]:
print("Confusion Matrix")
print('\n')
print(confusion_matrix(y_test, y_pred))
print('\n')
print("Classification Report")
print('\n')
print(classification_report(y_test, y_pred))

Confusion Matrix


[[ 513   19    0]
 [   9 1799    3]
 [   0   11  244]]


Classification Report


              precision    recall  f1-score   support

           0       0.98      0.96      0.97       532
           1       0.98      0.99      0.99      1811
           2       0.99      0.96      0.97       255

    accuracy                           0.98      2598
   macro avg       0.98      0.97      0.98      2598
weighted avg       0.98      0.98      0.98      2598



In [ ]:
rf,rfacc= classifiers(x_train,y_train,  'randomforest')

randomforest


In [ ]:
y_pred = rf.predict(x_test)

In [ ]:
print("Accuracy : ", round(accuracy_score(Y_test, y_pred)*100, 2), "%")

Accuracy :  51.74 %


In [ ]:
print("Confusion Matrix")
print('\n')
print(confusion_matrix(y_test, y_pred))
print('\n')
print("Classification Report")
print('\n')
print(classification_report(y_test, y_pred))

Confusion Matrix


[[ 497   35    0]
 [   6 1804    1]
 [   1   14  240]]


Classification Report


              precision    recall  f1-score   support

           0       0.99      0.93      0.96       532
           1       0.97      1.00      0.98      1811
           2       1.00      0.94      0.97       255

    accuracy                           0.98      2598
   macro avg       0.99      0.96      0.97      2598
weighted avg       0.98      0.98      0.98      2598



In [ ]:
rf.predict(cv.transform(["Achieve carbon neutrality no later than 2050, U.S. corporate average fuel economy, combined car and truck fleet, miles per gallon"]))

array([1])